# Lorenzo Venieri 
April 2022

<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.877 · Análisis de sentimientos y textos</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencias de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>


# PAC 2: Análisis de sentimientos

En esta práctica revisaremos y aplicaremos los conocimientos aprendidos en las últimas semanas. Concretamente trataremos 4 temas.

1. **Detección de elementos de opiniones**: Identificación de targets, aspectos y opinion words.
2. **Detección de polaridad**: Extracción de polaridades de las opiniones.
3. **Clasificación**: clasificación de opiniones.
4. **Evaluación**: comparación de modelos.

También repasaremos los temas genéricos de obtención y preprocesamiento de datos textuales.

# 1. Detection of opinion elements

In [1]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=c08e3b77d5c6470fc21afb365c3ab883fe70d26c338af81c1e3e92653ab00f45
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import pandas as pd
import gensim
import nltk
import re
nltk.download('punkt')
import os
import wget
import gzip
import shutil
from nltk import pos_tag, word_tokenize
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#I changed the path to adapt it to my drive
hotel_opinions_path='/content/drive/MyDrive/Colab Notebooks/NLP/PRA2/data/hotel_opinions_data.csv'
hotel_opinions_data = pd.read_csv(hotel_opinions_path)
hotel_opinions_data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,reviewer_nationality,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,review
0,3 Boulevard Saint Michel 5th arr 75005 Paris F...,120,10/25/2015,8.0,Royal Saint Michel,Lebanon,3,1193,0,1,9.6,"[' Leisure trip ', ' Group ', ' Double Room ',...",648 day,48.852836,2.344080,l ascenseur No Positive
1,Wrights Lane Kensington and Chelsea London W8 ...,1172,8/27/2015,7.8,Holiday Inn London Kensington,Australia,18,5945,56,2,9.6,"[' Leisure trip ', ' Solo traveler ', ' Execut...",707 day,51.499981,-0.192879,Having to log in each day with a new login an...
2,Via Tarchetti 2 Milan City Center 20121 Milan ...,436,9/18/2016,8.4,NH Milano Touring,Qatar,24,4568,6,2,7.1,"[' Leisure trip ', ' Family with young childre...",319 day,45.476917,9.196665,the TV is crazy the hotel host a lot of natio...
3,Avinguda Catedral 7 Ciutat Vella 08002 Barcelo...,170,2/16/2016,8.9,Col n Hotel Barcelona,Ireland,3,1300,37,3,9.2,"[' Leisure trip ', ' Couple ', ' Standard Doub...",534 day,41.384961,2.175667,No complaints Excellent location very helpfu...
4,Wrights Lane Kensington and Chelsea London W8 ...,1172,6/5/2016,7.8,Holiday Inn London Kensington,United Kingdom,45,5945,12,12,8.8,"[' Leisure trip ', ' Group ', ' 2 rooms ', ' S...",424 day,51.499981,-0.192879,We booked an executive room with super king b...


In [5]:
# New label to group by good or bad reviews

hotel_opinions_data["sentiment"] = hotel_opinions_data["Reviewer_Score"].apply(lambda x: "good" if x > 5 else "bad")

# Selection of the most relevant columns

hotel_opinions_data = hotel_opinions_data[["review","reviewer_nationality", "sentiment"]]
hotel_opinions_data.head()

,review,reviewer_nationality,sentiment
0,l ascenseur No Positive,Lebanon,good
1,Having to log in each day with a new login an...,Australia,good
2,the TV is crazy the hotel host a lot of natio...,Qatar,good
3,No complaints Excellent location very helpfu...,Ireland,good
4,We booked an executive room with super king b...,United Kingdom,good


The dataset contains reviews about hotels.

- Download the model Word2Vec of the Google News dataset, with approximately 3M words:

https://github.com/eyaler/word2vec-slim/blob/master/GoogleNews-vectors-negative300-SLIM.bin.gz

We will use just 500k words because of RAM limits

In [ ]:
from gensim.models import KeyedVectors
google_path='/content/drive/MyDrive/Colab Notebooks/NLP/PRA2/GoogleNews-vectors-negative300-SLIM.bin.gz'
#'GoogleNews-vectors-negative300-SLIM.bin.gz'
wv = KeyedVectors.load_word2vec_format(google_path, binary=True, limit=500000) 


- Convert every review in a single text in lowercase

In [ ]:
comment_text = " ".join(hotel_opinions_data['review'].to_list()).lower()

print(comment_text[:100])

 l ascenseur no positive  having to log in each day with a new login and password to use the wifi   


We obtain the phrases of the reviews. These phrases will be candidates to be targets and aspects of the opinions.
To do so we use the attached model: model_phrases

In [ ]:
import gensim
from gensim.models.phrases import Phraser
from gensim.models import Phrases

text_stream=word_tokenize(comment_text)

model_phrases = Phrases.load("/content/drive/MyDrive/Colab Notebooks/NLP/PRA2/data/model_phrases")

In [ ]:
ph=model_phrases[text_stream]

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def get_np(candidate):
  np = ''
  tokens = candidate.split('_')
  tagged_tokens = pos_tag((tokens))
  PoS_initial = tagged_tokens[0][1][:2]
  PoS_final = tagged_tokens[-1][1][:2]
  if PoS_initial == 'NN':
    if len(tagged_tokens) > 1:
      if PoS_final == 'NN':
        np = candidate
      else:
        np = tokens[0]
    else:
      np = candidate
  else:
    if PoS_final == 'NN':
      np = tokens[-1]
  return np

Explication:

The argument (candidate) is splitted into tokens (just 1 token if the phrase is of a single word), then each token is tagged with its part of speach.
In the command "tagged_tokens[0][1][:2]" the first 0 selects the tuple (token, PoS) in the list [(token,PoS)], the index 1 selects the PoS, then with [:2] we select all the characters in the PoS string.
If its component are both nouns, the output of the function is the full input phrase. If just one of the tokens is a noun then it will be the output. If both are not nominal the function returns the empty string.

In [ ]:
sint_nom=[get_np(candidate) for candidate in ph]
sint_nom=[s for s in sint_nom if s!='']

In [ ]:
#store the processed comments in a file
with open('NPs_COMMENTS.txt', 'w') as f:
    for item in sint_nom:
        f.write("%s\n" % item)

In [ ]:
len(sint_nom) #before removing empty strings: >38000

14584

In [ ]:
with open('NPs_COMMENTS.txt') as nf:
     nps_comments = nf.read().split('\n')

- To obtain the targets of the opinions we get the 100 phrases more similar to the term "hotel" (for the Word2Vec embedding) that are in the opinions

In [ ]:
term='hotel'
w2v_tuples = []

feature_names = list(set(nps_comments))
missing_terms_w2v=[0]

for i in range(0, len(feature_names)):
    if feature_names[i] != term:
      try:
        similarity=wv.similarity(term, feature_names[i])
        if similarity > 0:
          w2v_tuples.append((feature_names[i], similarity))
      except:
        missing_terms_w2v.append(feature_names[i])

w2v_sorted_tuples = sorted(w2v_tuples, key=lambda tup: tup[1], reverse=True)
#print(w2v_sorted_tuples[:100])

labels = ['Term', 'w2v similarity']

df_similarity = pd.DataFrame.from_records(w2v_sorted_tuples, columns=labels)

print (df_similarity[:100])

print(f'Number of missing terms encountered: {len(missing_terms_w2v)}')

          Term  w2v similarity
0       hotels        0.770973
1        motel        0.661712
2   restaurant        0.572378
3          inn        0.559031
4    apartment        0.500656
..         ...             ...
95    property        0.297659
96  waterfront        0.294929
97      dinner        0.294829
98    building        0.294021
99    upstairs        0.293971

[100 rows x 2 columns]
Number of missing terms encountered: 313


- We can also find the targets by the Wu and Palmer distance between the first synset (.n.01) of each phrase and the synset hotel.n.01

In [ ]:
#Adjust the dowload of NLTK, only if the dowload of 'wordnet' doesn't work properly

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus import wordnet as wn

wup_tuples=[]
missing_terms=[]

for term in feature_names:
    try:
        term=term.strip().replace(' ', '_')
        sense_term=wn.synset(term+'.n.01')
        wup_tuples.append((term, sense_term.wup_similarity(wn.synset('hotel.n.01'))))
        #print("Wu and Palmer distance between '{}' and 'hotel(n.01)': ".format(term),
            #sense_term.wup_similarity(wn.synset('hotel.n.01')))
    except: 
        missing_terms.append(term)
        #print("The term '{}' is not in WordNet ".format(term))

wup_sorted_tuples=sorted(wup_tuples, key=lambda tup: tup[1], reverse=True)
labels2 = ['Term', 'WuP similarity']

df_wup = pd.DataFrame.from_records(wup_sorted_tuples, columns=labels2)

print(df_wup[1:51])

print(f'Number of terms missing from WordNet encountered: {len(missing_terms)}')

            Term  WuP similarity
1         hostel        0.941176
2            inn        0.941176
3       building        0.933333
4          motel        0.888889
5     restaurant        0.875000
6        theatre        0.875000
7          house        0.875000
8   architecture        0.875000
9            pub        0.823529
10          cafe        0.823529
11      hospital        0.823529
12        garage        0.823529
13       balcony        0.800000
14       stadium        0.800000
15        bridge        0.800000
16       shelter        0.800000
17         tower        0.800000
18      fountain        0.800000
19        palace        0.777778
20        castle        0.777778
21         duomo        0.777778
22       terrace        0.750000
23        cellar        0.750000
24          knob        0.750000
25         works        0.750000
26       barrier        0.750000
27     courtyard        0.750000
28         patio        0.750000
29      basement        0.750000
30        

- Let's compare the first 50 targets found with the two methods:

In [ ]:
set_wup=set([t[0] for t in wup_sorted_tuples[1:51]])
set_w2v=set([t[0] for t in w2v_sorted_tuples[:50]])

In [ ]:
set_wup-set_w2v

{'architecture',
 'barrier',
 'basement',
 'block',
 'bridge',
 'building',
 'castle',
 'cellar',
 'closet',
 'courtyard',
 'decoration',
 'duomo',
 'facility',
 'fountain',
 'foyer',
 'frame',
 'garage',
 'knob',
 'lavatory',
 'layer',
 'lobby',
 'office',
 'patio',
 'rail',
 'shelter',
 'stadium',
 'surface',
 'terrace',
 'theatre',
 'toilet',
 'tower',
 'wall',
 'window',
 'works'}

In [ ]:
set_w2v-set_wup

{'accommodation',
 'airport',
 'apartments',
 'bar',
 'bellmen',
 'cabin',
 'chambermaid',
 'concierge',
 'deluxe',
 'doorman',
 'downtown',
 'elevator',
 'guest',
 'guests',
 'honeymoon',
 'hospitality',
 'hotels',
 'lodge',
 'lounge',
 'luxury',
 'maid',
 'mall',
 'occupancy',
 'plush',
 'rental',
 'residence',
 'restaurants',
 'rooms',
 'salon',
 'spa',
 'suite',
 'taxi',
 'tourist',
 'tourists'}

#### Comments

From the comparison between the targets found exclusively by each method I would say that the targets found by the w2v model are more appropriate as we have a lot of terms that represent things in a hotel that a customer may have an opinion about. For instance: accommodation, bar, bellmen, doorman, hospitality, lodge, lounge, rooms, suite, spa...

We can also see that from all the candidates to be targets the wordnet model discards 1142 of them because they are not in its database, while the wv2 supermodel of GoogleNews discards only 313 of the candidates. This is a known problem with WordNet as it misses many of the less used and relatively new words.

- We find the aspects related to the target according to the Word2Vec model. We find the phrases semantically similar to the targets. 

In [ ]:
def aspects(target,candidates, thresh):
  aspects=[]
  for c in candidates:
    similarity=wv.similarity(target,c)
    if c!=target and similarity>thresh:
      aspects.append((c,similarity))
  return aspects

targets=df_similarity['Term'].tolist()
targets.append('hotel')

target_aspects={target:aspects(target,targets,0.35) for target in targets}

In [ ]:
target_aspects['hotel']

[('hotels', 0.77097285),
 ('motel', 0.66171205),
 ('restaurant', 0.57237756),
 ('inn', 0.55903065),
 ('apartment', 0.5006558),
 ('airport', 0.48113677),
 ('rooms', 0.47720072),
 ('hospitality', 0.4728384),
 ('spa', 0.47282827),
 ('luxury', 0.47001636),
 ('occupancy', 0.46986637),
 ('accommodation', 0.46953398),
 ('lodge', 0.4682071),
 ('room', 0.46567726),
 ('tourist', 0.46373928),
 ('penthouse', 0.4624656),
 ('apartments', 0.45567948),
 ('guests', 0.45522782),
 ('hostel', 0.44753048),
 ('bellmen', 0.44641432),
 ('concierge', 0.44381618),
 ('lounge', 0.43101066),
 ('palace', 0.42241666),
 ('doorman', 0.41629168),
 ('restaurants', 0.41617218),
 ('marina', 0.40823388),
 ('cafe', 0.4020607),
 ('mall', 0.3997244),
 ('pub', 0.3995665),
 ('taxi', 0.38674596),
 ('downtown', 0.3828734),
 ('house', 0.38227427),
 ('residence', 0.37999085),
 ('hospital', 0.37931976),
 ('chambermaid', 0.37897807),
 ('bathroom', 0.3781805),
 ('guest', 0.37624604),
 ('honeymoon', 0.3761602),
 ('bedroom', 0.37577185)

In [ ]:
target_aspects['restaurant']

[('hotels', 0.4206994),
 ('motel', 0.45492327),
 ('inn', 0.4916499),
 ('apartment', 0.37224898),
 ('spa', 0.39078528),
 ('lounge', 0.38224435),
 ('doorman', 0.3871286),
 ('restaurants', 0.7722894),
 ('marina', 0.39246947),
 ('cafe', 0.6767455),
 ('mall', 0.44909427),
 ('pub', 0.5730096),
 ('taxi', 0.35405153),
 ('downtown', 0.40200517),
 ('salon', 0.51383865),
 ('bar', 0.5545204),
 ('breakfast', 0.42089343),
 ('nightlife', 0.41031972),
 ('waiter', 0.560682),
 ('eateries', 0.69405955),
 ('supermarket', 0.4991633),
 ('showroom', 0.37602308),
 ('catering', 0.42562118),
 ('boutique', 0.40765363),
 ('bartender', 0.5419379),
 ('barman', 0.3909236),
 ('dinner', 0.44607872),
 ('buffet', 0.5079515),
 ('waitress', 0.5417483),
 ('kitchen', 0.5017215),
 ('terrasse', 0.39377174),
 ('bakery', 0.59665316),
 ('shops', 0.39018017),
 ('shop', 0.48757964),
 ('pubs', 0.374289),
 ('plaza', 0.35097906),
 ('caf', 0.5124783),
 ('ambiance', 0.353152),
 ('trendy', 0.36814985),
 ('patio', 0.44196337),
 ('waitres

In [ ]:
def Wordnet_filter(target, thresh):
  wup_tuples=[]
  missing_terms=[]

  for term,similarity in target_aspects[target]:
    try:
      term=term.strip().replace(' ', '_')
      sense_term=wn.synset(term+'.n.01')
      wup_similarity= sense_term.wup_similarity(wn.synset(target +'.n.01'))
      if (wup_similarity+similarity)/2>thresh: #or simply wup_similarity > thresh
        wup_tuples.append((term, (wup_similarity+similarity)/2)) #or (term, wup_similarity)
    except:
      missing_terms.append(term)
  return wup_tuples

Wordnet_filter('restaurant',0.35)

[('motel', 0.6163505249553256),
 ('inn', 0.6575896538355771),
 ('apartment', 0.5611244887113571),
 ('spa', 0.3828926384449005),
 ('lounge', 0.4688999520407783),
 ('doorman', 0.40409061155821147),
 ('marina', 0.5491759093368755),
 ('cafe', 0.8089609724633834),
 ('mall', 0.5186647799085169),
 ('pub', 0.6982695109703961),
 ('taxi', 0.42702576518058777),
 ('downtown', 0.3885025829076767),
 ('salon', 0.590252657731374),
 ('bar', 0.6302013905609356),
 ('waiter', 0.4908673151543266),
 ('supermarket', 0.5495816498994828),
 ('showroom', 0.5409527184332119),
 ('boutique', 0.5196162885741183),
 ('bartender', 0.493191166056527),
 ('barman', 0.41768401695622337),
 ('buffet', 0.5480933960746317),
 ('waitress', 0.47087414264678956),
 ('kitchen', 0.6038019271457897),
 ('bakery', 0.614116055400748),
 ('shop', 0.5771231551965077),
 ('plaza', 0.3519601182026022),
 ('patio', 0.5959816873073578),
 ('decor', 0.5104491313298543),
 ('meal', 0.3506308724482854),
 ('bars', 0.4551749991046058),
 ('grocery', 0.51

#### Comments

The aspects found for the targets are overall adequate. If we decrease the threshold we will find more candidates that are not really aspects of the selected target. 

For instance let's analyze the aspects found for the target 'restaurant': the noun phrases 'apartment', 'restaurants', 'marina', 'spa', 'mall', 'supermarket', 'taxi' and others maybe are related to the target but they are not really aspects of a restaurant. 

Using WordNet we can filter again the aspects found by our w2v model analyzing their (of one of their synset for simplicity) Wu and Palmer distance with the target and take only the candidates that have similarity above a desired threshold. We could also combine the two similarity distances calculated doing their arithmentic mean and compare it to the threshold.

Using this method we are left with words like 'lounge, cafe, salon, bar, waiter, bartender, barman, waiter, waitress, kitchen, meal, pizza, steak, food, dish' that are aspects of a restaurant, but we also still have some words that are not aspects of restaurant.

We have to be cautious using Wordnet because we have to consider different possible synset of the same word and also the fact that a lot of words may be missing from wordnet as we have seen in the previous exercise.


In [ ]:
'''If we are interested in finding the aspects that refer to the targets in each one of the reviews we can build the list 
of all the reviews and then store the noun phrases found in each one of the reviews in another list. 
Then we can adapt the procedure done above to the noun phrases in this list to find the aspects that refer to
the targets in each review.'''

#opinions=[word_tokenize(opinion.lower()) for opinion in hotel_opinions_data['review'].to_list()]
#aspectos=[[get_np(phrase) for phrase in model_phrases[opinion] if get_np(phrase)!=''] for opinion in opinions]

- We will see if the targets in the opinions differs between nationalities (e.g. if the reviewer is from UK or USA)

In [ ]:
nationality_opinions={}
nationalities=hotel_opinions_data['reviewer_nationality'].unique()
for n in nationalities:
  idx=hotel_opinions_data['reviewer_nationality']==n
  nationality_opinions[n]=' '.join(hotel_opinions_data[idx]['review'].to_list()).lower()

In [ ]:
nationality_targets={}
for n in nationality_opinions:
  phrases=model_phrases[word_tokenize(nationality_opinions[n])]
  targets=[get_np(candidate) for candidate in phrases]
  targets=[s for s in targets if s!='']
  nationality_targets[n]=set(targets)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
print('Number of targets that are encountered only in one of the two nationality opinions: ' , 
len(nationality_targets[' United Kingdom '].symmetric_difference(nationality_targets[' United States of America '])))

Number of targets that are encountered only in one of the two nationality opinions:  1540


In [ ]:
print('Number of targets encountered in both nationality opinions: ', 
len(nationality_targets[' United Kingdom '] & nationality_targets[' United States of America ']))

Number of targets encountered in both nationality opinions:  367


- There are a lot of noun phrases that are exclusive of one of the two nationalities analyzed

In [ ]:
print('Targets exclusive of UK: ', nationality_targets[' United Kingdom '].difference(nationality_targets[' United States of America ']))

Targets exclusive of UK:  {'ect', 'student', 'badges', 'dinner', 'i_ll', 'majority', 'criticisms', 'definite', 'barcelona', 'description', 'pillowcase', 'busiest', 'difficulty', 'louder', 'straightens', 'sausage', 'travellers', 'paper', 'position', 'courtesy', 'theme', 'partners', 'system', 'client', 'scratches', 'choose', 'cot', 'grimy', 'heat', 'include', 'cut', 'dont', 'buzzing', 'eye', 'guess', 'checked', 'threadbare', 'cities', 'cheap', 'metropole', 'met', 'gras', 'look', 'rank', 'paintwork', 'considerate', 'relative', 'girlfriend', 'inbetween', 'mouth', 'pots', 'uniqueness', 'effort', 'exsperience', 'hadn_t', 'remind', 'fitness', 'milk', 'bonus', 'contact', 'strange', 'grenfell', 'hostel', 'nevermind', 'stand', 'cards', 'speed', 'speaker', 'woken', 'bowels', 'ni', 'corridor', 'shoulder', 'dodgy', 'apple', 'revamp', 'lazy', 'teenage', 'trendy', 'awesome', 'baths', 'days', 'vienna', 'answerphone', 'love', 'trades', 'music', 'sick', 'company', 'advise', 'verge', 'cash', 'heating', '

In [ ]:
print('Targets exclusive of USA: ', nationality_targets[' United States of America '].difference(nationality_targets[' United Kingdom ']))

Targets exclusive of USA:  {'slide', 'folks', 'direction', 'macro', 'separation', 'promenade', 'bellmen', 'collections', 'rent', 'statue', 'propose', 'cake', 'hip', 'marina', 'champs', 'beverages', 'transfer', 'month', 'movement', 'deserves', 'chose', 'selections', 'layer', 'croissants', 'buildings', 'center', 'sweet', 'quarter', 'travelers', 'bulbs', 'linens', 'globe', 'mentions', 'readp', 'vicinity', 'outlet', 'waterloo', 'dire', 'nord', 'avaikability', 'proffessional', 'brisk', 'postage', 'peasy', 'def', 'trouble', 'maps', 'suitcase', 'justify', 'tenders', 'boats', 'sticky', 'inches', 'files', 'ripoff', 'breathing', 'scale', 'seine', 'humor', 'bourne', 'subway', 'opera', 'poter', 'gotten', 'google', 'matt', 'tend', 'candies', 'relaxed', 'pompidou', 'mark', 'vegetarians', 'blankets', 'blocks', 'emma', 'sooo', 'novotel', 'cocktails', 'seal', 'europe', 'guide', 'aluminum', 'sides', 'gathering', 'flight', 'yummy', 'cane', 'offerings', 'honors', 'workers', 'tie', 'rouge', 'uber', 'sinks'

# 2. Polarity detection

## 2.1 Obtaining and preprocessing of data

- We obtain 1000 tweets about #Ukranie with the tweepy library

In [ ]:
import tweepy
from tweepy import API
from tweepy import OAuthHandler

CONSUMER_KEY = ###
CONSUMER_SECRET = ###
ACCESS_KEY = ###
ACCESS_SECRET = ###

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True)


query = 'Ukranie'
max_tweets = 1000

In [ ]:
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]

In [ ]:
searched_tweets[0]

Status(_api=<tweepy.api.API object at 0x7faa9683c990>, _json={'created_at': 'Fri Apr 22 08:40:26 +0000 2022', 'id': 1517423066300698626, 'id_str': '1517423066300698626', 'text': '@scythesociety Deutschland könnte der Ukraine dafür benötigtes Material liefern z.B die eingelagerten Marder.\nDa kö… https://t.co/wB1XXTcNpl', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'scythesociety', 'name': 'Xel', 'id': 126903118, 'id_str': '126903118', 'indices': [0, 14]}], 'urls': [{'url': 'https://t.co/wB1XXTcNpl', 'expanded_url': 'https://twitter.com/i/web/status/1517423066300698626', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'de', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1517291344837988352, 'in_reply_to_status_id_str': '1517291344837988352', 'in_reply_to_user_id': 126903118, 'in_reply_to_us

In [ ]:
tweets_text_list=[]
for tweet in searched_tweets:
  tweets_text_list.append(tweet._json['text'])

In [ ]:
import json
with open('tweets_text_list.txt', 'w') as f:
    f.write(json.dumps(tweets_text_list))

In [ ]:
tweets_path='/content/drive/MyDrive/Colab Notebooks/NLP/PRA2/tweets_text_list.txt'

with open(tweets_path, 'r') as f:
    tweets_text_list = json.loads(f.read())

In [ ]:
tweets_text_list[1]

'@TurkishNews7 why greece not giving their s300 russian air defence systems? they send nothing to Ukranie... greece… https://t.co/Fr2VvJMwin'

- filtering out strange characters, emojis, urls and unifying the text

In [ ]:
import re

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [ ]:
def clean_text(text):
  text = emoji_pattern.sub(r'',text)
  text = re.sub(r'(\s)http\S+','', text)
  text = re.sub('[^a-zA-Z0-9]', ' ', text)
  text = re.sub(r'[#@]','', text)
  return text

clean_tweets = [clean_text(sentence) for sentence in tweets_text_list]

In [ ]:
clean_tweets[30:50]

[' Lisa40393817  lesiavasylenko 100 donablo to Ukranie ppls under war situation Banksy style 3d aretworks porweder by ',
 ' oldmordim 100  donable to Ukranie ppls under war situation  under Bansky style  3d artworks  NFTsaints powered by ',
 'More of 100 resisters follow this account  Anti Biden  anti Ukranie  Maybe you got confused because her Twitter han ',
 ' denvergrl303  nathanjbridges  tariqnasheed  WithUcranie under Banksy style 3D ccryptoart 100  donable to Ukranie p ',
 'RT  Comandante Sur   Sanchez el  socialista  amigo que le regala armas al criminal se  ZelenskyWarCriminal  la  Ukranie que ha ilegalizado ',
 ' MahmouSewar 100  donable to Ukranie ppls under war situation',
 '                                                                                          WeSupportMEK ',
 ' Vick top55  carlesenric  EmbajadaRusa  KremlinRussia C Hope not  They are spanish  nato  ships  And that means tot ',
 ' elonmusk Going to ukranie',
 ' EmiliaKaminska  OnetWiadomosci zobacz materi

In [ ]:
tweets_text_list[30:50]

['@Lisa40393817 @lesiavasylenko 100%donablo to Ukranie ppls under war situation\nBanksy style 3d aretworks porweder by… https://t.co/OwH7JzQm2g',
 '@oldmordim 100% donable to Ukranie ppls under war situation. under Bansky style, 3d artworks #NFTsaints powered by… https://t.co/lk8vErRXud',
 'More of 100 resisters follow this account! Anti Biden, anti Ukranie! Maybe you got confused because her Twitter han… https://t.co/7W5ng6lLyR',
 '@denvergrl303 @nathanjbridges @tariqnasheed #WithUcranie under Banksy style 3D ccryptoart 100% donable to Ukranie p… https://t.co/uSf6kJxEw4',
 'RT @Comandante_Sur: #Sanchez el "socialista" amigo que le regala armas al criminal se #ZelenskyWarCriminal, la #Ukranie que ha ilegalizado…',
 '@MahmouSewar 100% donable to Ukranie ppls under war situation https://t.co/h5GFkWd2iQ',
 '🌐 مهم\u200cترین اخبار ایران و جهان در ۶۰ ثانیه\n🔸️۵شنبه اول اردیبهشت \u200c۱۴۰۱\n🔹️۲۱ آوریل ۲۰۲۲\n#خبر۶۰\n#WeSupportMEK… https://t.co/rpGXaFUAWm',
 '@Vick_top55 @carlesenric @EmbajadaR

## 2.2. Sentiment Analysis

- With the dictionary of opinion words (AFINN-111) we get the polarity of each tweet as the mean of the opinion words of the tweet

I created the sentword dictionary with all the opinion words in AFINN as keys and their polarity value as values. 

The function mean_opinions takes as argument a text and finds all the words in AFINN that are present in our text, sums the polarity value of these words and then divides this value by how many words we encountered. With this procedure we obtain the mean of the polarity values of the opinion words in the input text.

tweets_polarity is the list built by applying the mean_opinions function on every tweet in the list of tweets that we have preprocessed in the last exercise.

In [ ]:
AFINN_path='/content/drive/MyDrive/Colab Notebooks/NLP/PRA2/data/AFINN-111.txt'

sentword = {}

with open(AFINN_path, 'r') as snf:
    senlines = snf.readlines()
    for sl in senlines:
        sl = sl.strip().split('\t')
        sentword[sl[0]] = int(sl[1])

def mean_opinions(text):
    opinions_sum=0
    count=0
    tokens = word_tokenize(text.lower())
    for t in tokens:
        if re.match("^[a-z]+.*",t):
            if t in sentword:
                opinions_sum+=sentword[t]
                count+=1
    if count==0:
        return 0
    mean=opinions_sum/count
    return mean

tweets_polarity=[mean_opinions(tweet) for tweet in clean_tweets]
tweets_polarity[:10]

[-3.0, 0, -0.4, -3.0, -4.0, 0, 0, 0, 0, 0]

- The emoji-polarity file contains a document similar to AFINN-111 but for emojis. We will use it to get the polarity of the tweets containing emojis

In [ ]:
import json
import codecs
emoji_pol_path='/content/drive/MyDrive/Colab Notebooks/NLP/PRA2/data/emoji-polarity.txt'

data = json.load(codecs.open(emoji_pol_path, 'r', 'utf-8-sig'))
data["emojis"][1]['emoji']

'😠'

In [ ]:
emojis_polarity={data['emojis'][i]['emoji']:data['emojis'][i]['polarity'] for i in range(len(data['emojis']))}

In [ ]:
def mean_emojis(text):
    emojis_list=[]
    opinions_sum=0
    count=0
    tokens = word_tokenize(text.lower())
    for t in tokens:
      if re.match("^[a-z]+.*",t):
            if t in sentword:
                opinions_sum+=sentword[t]
                count+=1
      if t in emojis_polarity:
        emojis_list.append(t)
        opinions_sum+=emojis_polarity[t]
        count+=1
    if count==0:
        return 0, emojis_list
    mean=opinions_sum/count
    return mean, emojis_list #i made it return also the list of the emojis to check if it works properly

tweets_polarity_emojis=[mean_emojis(tweet)[0] for tweet in tweets_text_list]

In [ ]:
import numpy as np
difference=np.array(tweets_polarity_emojis)-np.array(tweets_polarity)

print(f'There are {np.count_nonzero(difference)} tweets that have changed their polarity considering also the emojis')
print(f'Of these tweets, the mean value of the difference in polarity is: {sum(difference)/np.count_nonzero(difference)}')
print(f'The mean over all the tweets is {sum(difference)/len(difference)}')

There are 41 tweets that have changed their polarity considering also the emojis
Of these tweets, the mean value of the difference in polarity is: 0.9186991869918698
The mean over all the tweets is 0.038992408557625945


#### Comments:

Considering the emojis, the polarity of the tweets increases by slightly less than 1 point.
It must be considered that we have encountered emojis (present in our dictionary) just in 41 tweets of the 1000 searched, so it may be not really significative, as the mean over all the tweets suggest.

# 3. Classification


In [ ]:
hotel_opinions_data = pd.read_csv(hotel_opinions_path) 

hotel_opinions_data["sentiment"] = hotel_opinions_data["Reviewer_Score"].apply(lambda x: "bad" if x < 5 else "good")

hotel_opinions_data = hotel_opinions_data[["review","reviewer_nationality", "sentiment"]]
hotel_opinions_data.head()

,review,reviewer_nationality,sentiment
0,l ascenseur No Positive,Lebanon,good
1,Having to log in each day with a new login an...,Australia,good
2,the TV is crazy the hotel host a lot of natio...,Qatar,good
3,No complaints Excellent location very helpfu...,Ireland,good
4,We booked an executive room with super king b...,United Kingdom,good


In [ ]:
hotel_opinions_data.isnull().sum().sum() #there are no NaN values

0

The reviews are already stripped of punctuation so there is no need to do it in preprocessing

In [ ]:
#I look at some opinions to see if the labeling good/bad is done right
hotel_opinions_data[hotel_opinions_data['sentiment']=='bad']['review'].iloc[3]

' Evelein with short blonde hair Sana thin with short black hair at front desk are rude no social skills ill mannered discourteous seems like they don t like the guests asking them questions like where the train station is or how to go to tourist places etc In addition to that the chambermaid does not replenish the soap cosmetic vanity set cotton buds cotton pads emery board dental set and the face towel We have to ring the 2424 number to ask for these basic standard supplies It would be quite helpful if the staff personnel specially the front desk reception are fully well trained to receive incoming guests in a pleasant warm and welcoming manner They should have the utmost politeness and patience and also professionalism in dealing with all kinds of queries that the guest may have specially for tourists which is also our first time to visit in Amsterdam I hope the management takes this into serious account and take important note on this matter Good day and thank you   Hotel itself is

- We vectorize the opinions with TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X=hotel_opinions_data['review'].values
y=hotel_opinions_data['sentiment'].values

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf.A, y, test_size=0.2, random_state=1
)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

from sklearn.metrics import accuracy_score 

pred = lr.predict(X_test)
# Let's see how the model performs on the test set
score = accuracy_score(y_test, pred)
print('The accuracy score on the test set is: {}'.format(score))

The accuracy score on the test set is: 0.79


In [ ]:
print(f'The model classifies the review "{X[917]}" as {lr.predict(X_tfidf[917])}')

The model classifies the review " The sauna is a joke only 70C no changing rooms There were hairs on my bed and they weren t mine Im the bathroom the front part of the counter had fallen off No free wifi  nice to be on the waterfront" as ['bad']


In [ ]:
from sklearn import metrics
cr = metrics.classification_report(y_test, pred)
print(cr)

              precision    recall  f1-score   support

         bad       0.81      0.54      0.65        72
        good       0.78      0.93      0.85       128

    accuracy                           0.79       200
   macro avg       0.80      0.74      0.75       200
weighted avg       0.79      0.79      0.78       200



# 4. Evaluation and comparison of the models

We will try with others models or other preprocessing steps to try to improve the performance

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

def get_wn_pos(pos):
    if re.match(r'^N',pos):
        wn_pos = 'n'
    elif re.match(r'^V',pos):
        wn_pos = 'v'
    else:
        wn_pos = 'n' 
    return wn_pos

def wnlemmatize(t):
    lemma = t
    lem = WordNetLemmatizer()
    postag = nltk.pos_tag([t])
    lemma = lem.lemmatize(t,get_wn_pos(postag[0][1]))
    return lemma
  

lem=WordNetLemmatizer()
reviews_tokenized=[word_tokenize(review) for review in X]
reviews_lemmatized=[[wnlemmatize(token) for token in review] for review in reviews_tokenized]
X_lemmatized=[' '.join(review) for review in reviews_lemmatized]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#cells below to see if it worked:

In [ ]:
diff_indexes=[i for i in range(len(reviews_tokenized[1])) if reviews_tokenized[1][i]!=reviews_lemmatized[1][i]]

In [ ]:
[reviews_tokenized[1][i] for i in diff_indexes]

['was',
 'mins',
 'was',
 'was',
 'sparkling',
 'bottled',
 'provided',
 'replenished']

In [ ]:
[reviews_lemmatized[1][i] for i in diff_indexes]

['be', 'min', 'be', 'be', 'sparkle', 'bottle', 'provide', 'replenish']

In [ ]:
#filter out the stopwords
tfidf_vectorizer_nostw = TfidfVectorizer(stop_words='english', lowercase=False)
X_tfidf_2 = tfidf_vectorizer_nostw.fit_transform(X_lemmatized)

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    X_tfidf_2.A, y, test_size=0.2, random_state=1
)

lr2 = LogisticRegression()
lr2.fit(X_train_2, y_train_2)

pred2 = lr2.predict(X_test_2)
# Let's see how the model performs on the test set
score2 = accuracy_score(y_test_2, pred2)
print('The accuracy score on the test set is: {}'.format(score2))

The accuracy score on the test set is: 0.81


In [ ]:
cr2 = metrics.classification_report(y_test_2, pred2)
print('Logistic regression with lemmatization and without stopwords:')
print(cr2)
print('')
print('Logistic regression with no preprocessing:')
print(cr)

Logistic regression with lemmatization and without stopwords:
              precision    recall  f1-score   support

         bad       0.89      0.54      0.67        72
        good       0.79      0.96      0.87       128

    accuracy                           0.81       200
   macro avg       0.84      0.75      0.77       200
weighted avg       0.82      0.81      0.80       200


Logistic regression with no preprocessing:
              precision    recall  f1-score   support

         bad       0.81      0.54      0.65        72
        good       0.78      0.93      0.85       128

    accuracy                           0.79       200
   macro avg       0.80      0.74      0.75       200
weighted avg       0.79      0.79      0.78       200



All metrics improved slightly (~ +0.02)

In [ ]:
from sklearn.svm import LinearSVC

classifier = LinearSVC()

svm_model = classifier.fit(X=X_train, y=y_train)

y_pred = svm_model.predict(X_test)
score3 = accuracy_score(y_test, y_pred)
print('The accuracy score on the test set is: {}'.format(score3))
cr3 = metrics.classification_report(y_test, y_pred)
print('')
print('LinearSVC without lemmatization:')
print(cr3)

The accuracy score on the test set is: 0.825

LinearSVC without lemmatization:
              precision    recall  f1-score   support

         bad       0.79      0.69      0.74        72
        good       0.84      0.90      0.87       128

    accuracy                           0.82       200
   macro avg       0.82      0.80      0.80       200
weighted avg       0.82      0.82      0.82       200



In [ ]:
classifier = LinearSVC()

svm_model = classifier.fit(X=X_train_2, y=y_train_2)

y_pred_2 = svm_model.predict(X_test_2)
score4 = accuracy_score(y_test_2, y_pred_2)
print('The accuracy score on the test set is: {}'.format(score4))
cr4 = metrics.classification_report(y_test_2, y_pred_2)
print('')
print('LinearSVC with lemmatization:')
print(cr4)

The accuracy score on the test set is: 0.81

LinearSVC with lemmatization:
              precision    recall  f1-score   support

         bad       0.77      0.68      0.72        72
        good       0.83      0.88      0.86       128

    accuracy                           0.81       200
   macro avg       0.80      0.78      0.79       200
weighted avg       0.81      0.81      0.81       200



In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=10)

rf_model = classifier.fit(X=X_train, y=y_train)

y_pred_5 = rf_model.predict(X_test)

cr5 = metrics.classification_report(y_test, y_pred_5)
print('RandomForest Classifier:')
print(cr5)

RandomForest Classifier:
              precision    recall  f1-score   support

         bad       0.60      0.44      0.51        72
        good       0.73      0.84      0.78       128

    accuracy                           0.69       200
   macro avg       0.67      0.64      0.65       200
weighted avg       0.68      0.69      0.68       200



We obtain the best performance using the LinearSVC model without lemmatizing the input. So I would say that good results depend more on the choice of the right model than on the preprocessing steps.